In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_parquet("D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### optimize for memory

In [4]:
df.select_dtypes('float64').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 12 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   passenger_count        float64
 1   trip_distance          float64
 2   RatecodeID             float64
 3   fare_amount            float64
 4   extra                  float64
 5   mta_tax                float64
 6   tip_amount             float64
 7   tolls_amount           float64
 8   improvement_surcharge  float64
 9   total_amount           float64
 10  congestion_surcharge   float64
 11  airport_fee            float64
dtypes: float64(12)
memory usage: 225.6 MB


In [5]:
df.select_dtypes('float64').max()

passenger_count               9.00
trip_distance            306159.28
RatecodeID                   99.00
fare_amount              401092.32
extra                        33.50
mta_tax                      16.59
tip_amount                  888.88
tolls_amount                193.30
improvement_surcharge         0.30
total_amount             401095.62
congestion_surcharge          2.50
airport_fee                   1.25
dtype: float64

In [6]:
df.select_dtypes('int64').max()

VendorID          6
PULocationID    265
DOLocationID    265
payment_type      5
dtype: int64

In [7]:
new_types = dict(
    float16 = ['congestion_surcharge', 'airport_fee', 'improvement_surcharge', 'mta_tax', 'extra', 'passenger_count', 'tip_amount', 'tolls_amount', 'RatecodeID'],
    float32 = ['trip_distance', 'fare_amount', 'total_amount'],
    int16= ['PULocationID', 'DOLocationID'],
    int8 = ['VendorID', 'payment_type']
)

In [8]:
for key in new_types:
    for col in new_types[key]:
        df[col] = df[col].astype(key)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int8          
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float16       
 4   trip_distance          float32       
 5   RatecodeID             float16       
 6   store_and_fwd_flag     object        
 7   PULocationID           int16         
 8   DOLocationID           int16         
 9   payment_type           int8          
 10  fare_amount            float32       
 11  extra                  float16       
 12  mta_tax                float16       
 13  tip_amount             float16       
 14  tolls_amount           float16       
 15  improvement_surcharge  float16       
 16  total_amount           float32       
 17  congestion_surcharge   float16       
 18  airport_fee           

In [10]:
def optimeToFitMemory(df):
    new_types = dict(
        float16 = ['congestion_surcharge', 'airport_fee', 'improvement_surcharge', 'mta_tax', 'extra', 'passenger_count', 'tip_amount', 'tolls_amount', 'RatecodeID'],
        float32 = ['trip_distance', 'fare_amount', 'total_amount'],
        int16= ['PULocationID', 'DOLocationID'],
        int8 = ['VendorID', 'payment_type']
    )
    for key in new_types:
        for col in new_types[key]:
            df[col] = df[col].astype(key)
    return df

In [11]:
### merge memory optimized dataframes
files = [
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-01.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-02.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-03.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-04.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-05.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-06.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-07.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-08.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-09.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-10.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-11.parquet",
    "D:\\uber-taxi-demand\\data\\yellow_tripdata_2022-12.parquet",

]
optimized_df = None
for file in files:
    df = pd.read_parquet(file)
    optimized_df = pd.concat([optimized_df, optimeToFitMemory(df)])

D:\timeseriesEnv\lib\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [12]:
optimized_df.shape

(39656098, 19)

In [13]:
optimized_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.650391,0.0,0.300049,21.950001,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.000000,0.0,0.300049,13.300000,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.759766,0.0,0.300049,10.560000,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.000000,0.0,0.300049,11.800000,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.000000,0.0,0.300049,30.299999,2.5,0.0


In [14]:
optimized_df.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
3399544,2,2022-12-31 23:46:00,2023-01-01 00:11:00,NaN,11.56,NaN,None,16,36,0,39.549999,0.0,0.5,8.210938,0.0,1.0,49.259998,NaN,NaN
3399545,2,2022-12-31 23:13:24,2022-12-31 23:29:08,NaN,5.06,NaN,None,75,50,0,26.230000,0.0,0.5,0.000000,0.0,1.0,30.230000,NaN,NaN
3399546,2,2022-12-31 23:00:49,2022-12-31 23:26:57,NaN,13.35,NaN,None,168,197,0,47.730000,0.0,0.5,9.851562,0.0,1.0,59.080002,NaN,NaN
3399547,1,2022-12-31 23:02:50,2022-12-31 23:16:05,NaN,0.00,NaN,None,238,116,0,12.740000,0.0,0.5,0.000000,0.0,1.0,16.740000,NaN,NaN
3399548,2,2022-12-31 23:00:15,2022-12-31 23:22:56,NaN,4.24,NaN,None,85,37,0,19.600000,0.0,0.5,4.000000,0.0,1.0,25.100000,NaN,NaN


In [15]:
optimized_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39656098 entries, 0 to 3399548
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int8          
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float16       
 4   trip_distance          float32       
 5   RatecodeID             float16       
 6   store_and_fwd_flag     object        
 7   PULocationID           int16         
 8   DOLocationID           int16         
 9   payment_type           int8          
 10  fare_amount            float32       
 11  extra                  float16       
 12  mta_tax                float16       
 13  tip_amount             float16       
 14  tolls_amount           float16       
 15  improvement_surcharge  float16       
 16  total_amount           float32       
 17  congestion_surcharge   float16       
 18  airport_fee            flo

In [16]:
optimized_df.isnull().sum()

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count          1368303
trip_distance                  0
RatecodeID               1368303
store_and_fwd_flag       1368303
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     1368303
airport_fee              1368303
dtype: int64

In [22]:
optimized_df.to_csv('D:\\uber-taxi-demand\\data\\yellow_tripdata_2022.csv', index=False)